# Einstieg Selenium

In dieser Stunde schauen wir uns zum Einstieg wieder die Seite mit den Zitaten an. Allerdings widmen wir uns der Version der Seite https://quotes.toscrape.com/js/. Es handelt sich dabei um eine Version der Seite, bei der die Zitate mithilfe von JavaScript dargestellt werden. Zum Einstieg führen wir ein kleines Experiment durch. Öffnet euer Jupyter Notebook aus dem Abschnitt "Einstieg BeautifulSoup". Ersetzt URL = "https://quotes.toscrape.com" durch URL = "https://quotes.toscrape.com/js" und führt alle Codezellen aus. Was passiert?

Woran liegt das? Die Inhalte der Seite werden offensichtlich nicht vorgeladen, sondern sie werden erst beim Seitenaufruf ergänzt. Das könnt ihr nachvollziehen, indem ihr die Seite im Browser aufruft, die Entwicklertools öffnet, JavaScript deaktiviert, und die Seite https://quotes.toscrape.com/js/ neu ladet. Welche Inhalte der Seite werden beim neu Laden angezeigt? Wie ist das im Vergleich mit der Seite https://quotes.toscrape.com/?

### Was ist Selenium?

Zum Scrapen solcher Inhalte können wir BeautifulSoup nicht verwenden, weil BeautifulSoup mit solchen mithilfe von JavaScript dargestellten Inhalten nicht umgehen kann: In unserem quotes.toscrape-Webscraper haben wir ja nur eine HTTP-Anfrage gestellt, und diese Anfrage liefert nur die HTML-Dokumente, die bereits auf dem Server liegen, bevor wir die Webseite aufrufen. Wir müssen also irgendwie an die Inhalte kommen, welche mithilfe von JavaScript im Client nachträglich eingefügt werden.

Dazu gibt es verschiedene spezialisierte Python-Pakete und Bibliotheken, am gängigsten sind Selenium, die leichtgewichtigere Selenium-Variante Helium sowie Puppeteer und Playwright. Daneben gibt es auch Splash, das in Kombination mit Scrapy, eine BeautifulSoup-Alternative, verwendet wird. Das wohl meist verwendete Paket zum Scrapen dynamischer Inhalte war im letzten Jahrzehnt immer Selenium, allerdings wird es vermehrt durch Playwright verdrängt. Zum Verständnis von Playwright müssten wir uns aber noch ein anderes Netzwerkprotokoll ansehen, nämlich WebSocket. Im Rahmen dieses Seminars werden wir deswegen bei HTTP bleiben und lernen anstelle von Playwright das trotz der Konkurrenz immer noch etablierte Selenium kennen.

Aber was ist denn jetzt Selenium?

> Selenium automates browsers. That's it! What you do with that power is entirely up to you. Primarily it is for automating web applications for testing purposes, but is certainly not limited to just that.

Quelle: [Selenium 2023](https://www.selenium.dev/)

Selenium ist ein Projekt, das verschiedene Bibliotheken in verschiedenen Programmiersprachen umfasst und ursprünglich zum Testen von Webseiten entwickelt wurde. **Ganz allgemein ermöglich es Selenium, Webbrowser automatisiert zu steuern und mit ihnen zu interagieren. Das heißt, dass wir im Rahmen von Web Scraping mithilfe von Selenium auf die gerenderte Version einer Webseite zugreifen und direkt aus unserem Python-Notebook heraus verschiedene Aktionen im Webbrowser ausführen können, z.B. das Öffnen von URLs, Suchen nach und Klicken auf Elemente, Ausfüllen von Formularen und Extrahieren von Daten.** Um Selenium verwenden zu können, brauchen wir zusätzlich einen Browser und einen Treiber für diesen Browser (=Webdriver). In unserem Beispiel heute werden wir Selenium im Zusammenhang mit Chrome und dem [Chrome WebDriver](https://sites.google.com/chromium.org/driver/) verwenden. Der Chrome WebDriver stellt die Verbindung zum Chrome-Browser her und ermöglicht die Kommunikation zwischen Selenium und dem Browser.


### Beispiel: Zitate mit Python Selenium scrapen

Als Beispiel sehen wir uns zunächst wieder den Code von jemand anderem an, nämlich von [Lewis Kori](https://lewiskori.com/blog/beginner-s-guide-to-web-scraping-with-python-s-selenium/).

Wir nehmen allerdings zwei Änderungen an dessen Code vor: Wir installieren den ChromeDriver nicht manuell, sondern mithilfe des Pakets webdriver-manager. Dadurch müssen wir beim Aufruf der Funktion Chrome() aus dem Paket selenium nicht manuell den Pfad zum installierten ChromeDriver angeben, sondern der Pfad wird aufomatisch eingesetzt.

Außerdem verwenden wir den Präfix https anstelle von http für die quotes.toscrape Website, um eine verschlüsselte Verbindung zu der Seite herzustellen.


In [ ]:
# import sys
# !conda install --yes --prefix {sys.prefix} selenium
# !conda install --yes --prefix {sys.prefix} webdriver-manager

In [ ]:
# !conda list # selenium version 3

In [ ]:
# Code für Python-selenium Version 3: https://github.com/SergeyPirogov/webdriver_manager
# from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager
# import pandas as pd

# driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
# pages = 11

# total = []
# for page in range(1,pages):

#     url = "https://quotes.toscrape.com/js/page/" + str(page) + "/"

#     driver.get(url)

#     quotes = driver.find_elements_by_class_name("quote")
#     for quote in quotes:
#         quote_text = quote.find_element_by_class_name('text').text[1:-2]
#         author = quote.find_element_by_class_name('author').text
#         new = ((quote_text,author))
#         total.append(new)

# driver.close()
# df = pd.DataFrame(total,columns=['quote','author'])
# df.to_csv('quoted.csv')

Zur Übung sollt ihr euch im Rest der Stunde selbst erarbeiten, was dieses Skript macht. Schaut dazu in die Dokumentationsseiten zum Selenium-Python-Paket:  https://selenium-python.readthedocs.io/. Kommentiert jede Zeile, damit ihr euch später noch an die Ergebnisse eurer Recherche erinnern könnt: Diese Vorarbeit braucht ihr für die Übungsaufgabe!

### Quellen

```{bibliography}
   :list: enumerated
   :filter: keywords % "selenium"
```